In [1]:
from scipy.ndimage import gaussian_filter
import matplotlib.pyplot as plt
import numpy as np
import rupho as rp
import pylab as pl
%matplotlib qt

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in ""


In [2]:
def generate_terrain_volume(size=100, scale=1.0, smooth=3):
    """
    Generate a 3D binary volume (n x n x n) based on a 2D terrain.
    Voxels below the terrain surface are filled (value=1), others are empty (value=0).
    """
    # Generate smoothed 2D terrain
    x = np.linspace(0, 1, size)
    y = np.linspace(0, 1, size)
    X, Y = np.meshgrid(x, y)
    noise = np.random.rand(size, size)
    terrain = gaussian_filter(noise, sigma=smooth) * scale

    # Normalize terrain to voxel grid height range
    terrain_normalized = (terrain - terrain.min()) / (terrain.max() - terrain.min())
    terrain_voxels = (terrain_normalized * (size - 1)).astype(int)

    # Create 3D binary volume
    volume = np.zeros((size, size, size), dtype=np.uint8)
    for i in range(size):
        for j in range(size):
            h = terrain_voxels[i, j]
            volume[i, j, :h+1] = 1  # fill below terrain

    return volume, X, Y, terrain


In [3]:
def fractal_dimension(volume):
    # finding all the non-zero pixels
    voxels = []
    Lx, Ly, Lz = volume.shape
    
    for i in range(Lx):
        for j in range(Ly):
            for k in range(Lz):
                if volume[i, j, k]>0:
                    voxels.append((i, j, k))
    
   
    print("Dimensions:", Lx, Ly, Lz)
    voxels = np.array(voxels)
    print("Number of non-zero voxels:", voxels.shape[0])
    
    # computing the fractal dimension
    # considering only scales in a logarithmic list
    scales=np.logspace(0.01, 1, num=10, endpoint=False, base=2)
    Ns=[]
    # looping over several scales
    for scale in scales:
        print (scale)
        # computing the histogram
        H, edges = np.histogramdd(voxels, bins=(np.arange(0, Lx, scale), np.arange(0, Ly, scale), np.arange(0, Lz, scale)))
        Ns.append(np.sum(H>0))
    
    # linear fit, polynomial of degree 1
    coeffs=np.polyfit(np.log(scales), np.log(Ns), 1)
    return -coeffs[0], coeffs, scales, Ns

In [ ]:
volume, X, Y, Z = generate_terrain_volume(size=100, scale=3, smooth=10)

D, coeffs, scales, Ns = fractal_dimension(volume)
print("The Hausdorff dimension is ",D)
fig = plt.figure(figsize=(10,5))

ax1 = fig.add_subplot(121, projection='3d')
ax1.plot_surface(X, Y, Z, cmap='terrain', linewidth=0, antialiased=False)
ax1.set_title("Random Terrain")
ax1.set_xlabel("X")
ax1.set_ylabel("Y")
ax1.set_zlabel("Z")
ax1.set_zlim(1,2)

ax2 = fig.add_subplot(122)
ax2.plot(np.log(scales), np.polyval(coeffs, np.log(scales)))
ax2.plot(np.log(scales),np.log(Ns), 'o', color="red", markerfacecolor="none")
ax2.set_title("Linear regression")
ax2.set_xlabel(r'log $\epsilon$')
ax2.set_ylabel(r'log N')
ax2.grid()

plt.tight_layout()
plt.show()


Dimensions: 100 100 100
Number of non-zero voxels: 492390
1.0069555500567189
1.0784804316944516
1.1550857845535842
1.2371324786871727
1.3250070170452075
1.4191233562003824
1.5199248564015158
1.6278863701408692
1.7435164790741244
1.8673598898306263
The Hausdorff dimension is  2.959541899281039
